In [2]:
import os
from pprint import pprint
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import cv2
from PIL import Image
import keras

In [3]:
from homomorphicfilter import HomomorphicFilter

In [4]:
homomorphicfilter = HomomorphicFilter(a = 0.75, b = 1.25)

In [5]:
model_path = "models/emotion_model.hdf5"

In [6]:
model = keras.models.load_model(model_path)

In [7]:
label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
label_indices = list(range(0, 7))
id_name_map = dict(zip(label_indices, label_names))

In [8]:
def filterd_and_normalized(img, homomorphicfilter):
    img_filtered = homomorphicfilter.filter(I=img, filter_params=[30,2])
    img_normalized = cv2.equalizeHist(img_filtered)
    return img_normalized

In [9]:
def load_image_as_grayscale_and_apply_filter_and_normalization(img_path_in, homomorphicfilter):
    img = Image.open(img_path_in)
    img = img.convert("L")
    img = np.array(img)
    img_normalized = filterd_and_normalized(img, homomorphicfilter)
    return img_normalized

In [10]:
def reshape_for_model(img, dim):
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized.reshape(1, *(resized.shape), 1)

In [11]:
def predict_image(img_path_in, homomorphicfilter, model, id_name_map, dim):
    filteredandnormalized = load_image_as_grayscale_and_apply_filter_and_normalization(img_path_in, homomorphicfilter)
    model_in = reshape_for_model(filteredandnormalized, dim)
    preds = model.predict(model_in)
    max_ind = np.argmax(preds)
    return (max_ind, id_name_map[max_ind])

In [12]:
def test_model_accuracy(test_image_folder):
    path, dirs, files = next(os.walk(test_image_folder))
    results = np.ones((len(files),))
    for i, file in enumerate(files):
        img_path_in = os.path.join(path, file)
        max_ind, id_name_map[max_ind] = predict_image(img_path_in, homomorphicfilter, model, id_name_map, dim=(64, 64))
        results[i] = max_ind
    return results

In [ ]:
label_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
label_indices = list(range(0, 7))
id_name_map = dict(zip(label_indices, label_names))

In [15]:
emotion_index = 0
emotion_name = id_name_map[emotion_index]
test_image_folder = f"archive/test/{emotion_name.lower()}"

In [16]:
print(emotion_name)

'Angry'

In [ ]:
results = test_model_accuracy(test_image_folder)
accuracy = (results == emotion_index).sum()/(results.shape)

In [ ]:
print(f"Accuracy for {emotion_name} = {int(10000*accuracy[0])/100.0}%")

# Test all labels

In [ ]:
accuracy_dict = {}

In [ ]:

for i, label_name in enumerate(label_names):
    test_image_folder = f"archive/test/{label_name.lower()}"
    results = test_model_accuracy(test_image_folder)
    accuracy = (results == i).sum()/(results.shape)
    accuracy_dict[label_name] = int(10000*accuracy[0])/100.0
    print(f"Completed the label {label_name}")

In [ ]:
pprint(accuracy_dict)